In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import duckdb as db
import plotly.express as px


ACTEURS SELON TYPE DE FILMS

In [2]:
def sql(query, database = "../data/database.duckdb"):
    con = db.connect(database)
    res = con.execute(query).df()
    con.close()
    return res

In [3]:
sql("show tables")

,name
0,dataframe_view
1,name_basics
2,title_akas
3,title_basics
4,title_crew
5,title_principals
6,title_ratings
7,tmdb


In [3]:
df_all = sql("SELECT * from dataframe_view")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [8]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101563 entries, 0 to 101562
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   tconst               101563 non-null  object 
 1   Title                101563 non-null  object 
 2   Genres               101563 non-null  object 
 3   Year                 101563 non-null  object 
 4   Runtime              101563 non-null  object 
 5   Actors               101563 non-null  object 
 6   Director             101563 non-null  object 
 7   Writers              99412 non-null   object 
 8   Resume               100412 non-null  object 
 9   Rating               101563 non-null  float64
 10  Votes                101563 non-null  int64  
 11  Budget               101563 non-null  int64  
 12  Revenue              101563 non-null  int64  
 13  Profit               101563 non-null  int64  
 14  ProductionCompanies  101563 non-null  object 
 15  ProductionCountri

In [4]:
data_copy = df_all

In [5]:
#CLEAN DATA 
def clean_row(row):
    return row.replace("[", "").replace("]", "").replace("'", "")
data_copy.drop(columns= ['Profit','Revenue', 'Budget'], inplace=True) #trop de valeurs manquantes, DF faussé
data_copy['ProductionCountries'] = data_copy['ProductionCountries'].apply(lambda x: 'Unknown' if x =='\\N' else x)
data_copy['Writers'] = data_copy['Writers'].apply(lambda x: 'Unknown' if x== '\\N'  else x)
data_copy['Resume'] = data_copy['Resume'].apply(lambda x: 'Unknown' if x == '\\N'  else x)
data_copy['Poster'] = data_copy['Poster'].apply(lambda x: 'Unknown' if x == '\\N'  else x)
#nettoyage de la colonne Year (il semblerait qu'il y ait qu'une ligne avec une valeur '\N')
data_copy['Year'] = data_copy['Year'].replace('\\N', np.nan)
data_copy = data_copy.dropna(subset=['Year'])
#On convertit la colonne Year en int pour pouvoir l'utiliser dans le modèle
data_copy['Year'] = data_copy['Year'].astype(int)
data_copy['Resume'].fillna('Unknown', inplace=True)
df_runtime = data_copy[data_copy['Runtime'] != '\\N']
df_runtime['Runtime'] = df_runtime['Runtime'].astype(int)
data_copy['Runtime'] = data_copy['Runtime'].apply(lambda x: df_runtime['Runtime'].median() if x == '\\N' else x)
data_copy['Runtime'] = data_copy['Runtime'].astype(int)
data_copy['Runtime'].describe()
data_copy['ProductionCountries'].fillna('Unknown', inplace=True)
data_copy['ProductionCountries'] = data_copy['ProductionCountries'].apply(clean_row)
data_copy['ProductionCompanies'] = data_copy['ProductionCompanies'].apply(clean_row)
data_copy['ProductionCountries'] = data_copy['ProductionCountries'].apply(
    lambda x: ', '.join(dict.fromkeys(str(x).split(', ')))
)
data_copy['Writers'].fillna('Unknown', inplace=True)


C:\Users\antoi\AppData\Local\Temp\ipykernel_98624\359270650.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_copy['Year'] = data_copy['Year'].astype(int)
C:\Users\antoi\AppData\Local\Temp\ipykernel_98624\359270650.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_copy['R

In [12]:
data_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101562 entries, 0 to 101562
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   tconst               101562 non-null  object 
 1   Title                101562 non-null  object 
 2   Genres               101562 non-null  object 
 3   Year                 101562 non-null  int64  
 4   Runtime              101562 non-null  int64  
 5   Actors               101562 non-null  object 
 6   Director             101562 non-null  object 
 7   Writers              101562 non-null  object 
 8   Resume               101562 non-null  object 
 9   Rating               101562 non-null  float64
 10  Votes                101562 non-null  int64  
 11  ProductionCompanies  101562 non-null  object 
 12  ProductionCountries  101562 non-null  object 
 13  Poster               99331 non-null   object 
dtypes: float64(1), int64(3), object(10)
memory usage: 11.6+ MB


In [44]:
data_copy['Genres'].value_counts().head(20)

Genres
Drama                      13854
Comedy                      7719
Comedy,Drama                4864
Drama,Romance               3924
Comedy,Drama,Romance        2872
Comedy,Romance              2794
Horror                      2671
Action,Crime,Drama          1727
Drama,Thriller              1629
Crime,Drama                 1582
Horror,Thriller             1506
Thriller                    1399
Documentary                 1368
Crime,Drama,Thriller        1327
Action,Drama                 987
Crime,Drama,Mystery          945
Drama,War                    862
Horror,Mystery,Thriller      830
Action                       815
Comedy,Horror                770
Name: count, dtype: int64

In [22]:
top_20_genres = data_copy['Genres'].value_counts().head(20).index.tolist()
top_20_genres

['Drama',
 'Comedy',
 'Comedy,Drama',
 'Drama,Romance',
 'Comedy,Drama,Romance',
 'Comedy,Romance',
 'Horror',
 'Action,Crime,Drama',
 'Drama,Thriller',
 'Crime,Drama',
 'Horror,Thriller',
 'Thriller',
 'Documentary',
 'Crime,Drama,Thriller',
 'Action,Drama',
 'Crime,Drama,Mystery',
 'Drama,War',
 'Horror,Mystery,Thriller',
 'Action',
 'Comedy,Horror']

In [6]:
#Création d'un  DF Exploded
df_exploded = data_copy.copy()

df_exploded['Genres'] = df_exploded['Genres'].str.split(',').apply(lambda x: [g.strip() for g in x])
df_exploded['Actors'] = df_exploded['Actors'].str.split(', ')

df_exploded = df_exploded.explode('Genres')
df_exploded = df_exploded.explode('Actors')

In [46]:
df_exploded.head()

,tconst,Title,Genres,Year,Runtime,Actors,Director,Writers,Resume,Rating,Votes,ProductionCompanies,ProductionCountries,Poster
0,tt0499549,Avatar,Action,2009,162,Joel David Moore,James Cameron,James Cameron,"In the 22nd century, a paraplegic Marine is di...",7.9,1431034,"Dune Entertainment, Lightstorm Entertainment, ...","US, GB",/jRXYjXNq0Cs2TcJjLkki24MLp7u.jpg
0,tt0499549,Avatar,Action,2009,162,Laz Alonso,James Cameron,James Cameron,"In the 22nd century, a paraplegic Marine is di...",7.9,1431034,"Dune Entertainment, Lightstorm Entertainment, ...","US, GB",/jRXYjXNq0Cs2TcJjLkki24MLp7u.jpg
0,tt0499549,Avatar,Action,2009,162,Michelle Rodriguez,James Cameron,James Cameron,"In the 22nd century, a paraplegic Marine is di...",7.9,1431034,"Dune Entertainment, Lightstorm Entertainment, ...","US, GB",/jRXYjXNq0Cs2TcJjLkki24MLp7u.jpg
0,tt0499549,Avatar,Action,2009,162,Zoe Saldaña,James Cameron,James Cameron,"In the 22nd century, a paraplegic Marine is di...",7.9,1431034,"Dune Entertainment, Lightstorm Entertainment, ...","US, GB",/jRXYjXNq0Cs2TcJjLkki24MLp7u.jpg
0,tt0499549,Avatar,Action,2009,162,CCH Pounder,James Cameron,James Cameron,"In the 22nd century, a paraplegic Marine is di...",7.9,1431034,"Dune Entertainment, Lightstorm Entertainment, ...","US, GB",/jRXYjXNq0Cs2TcJjLkki24MLp7u.jpg


In [ ]:
actor_genre_counts = df_exploded.groupby(['Genres', 'Actors']).size().reset_index(name='Count')
top_actors_per_genre = actor_genre_counts.sort_values(['Genres', 'Count'], ascending=[True, False])
top_actors_per_genre = top_actors_per_genre.groupby('Genres').head(5)
print(top_actors_per_genre)

         Genres                 Actors  Count
8550     Action           Brahmanandam    166
53558    Action            Prakash Raj    130
48631    Action                 Nassar    113
27553    Action            Jackie Chan     91
3139     Action       Amitabh Bachchan     86
...         ...                    ...    ...
1115130      \N            Prakash Raj      2
1114968      \N         Aahuthi Prasad      1
1114969      \N               Adam Lim      1
1114970      \N           Aindrita Ray      1
1114971      \N  Aishwariyaa Bhaskaran      1

[130 rows x 3 columns]


In [12]:

fig = px.bar(
    data_frame=top_actors_per_genre,
    x="Actors",
    y="Count",
    animation_frame= 'Genres',                   
    title="Acteurs les plus présents par genre",
    barmode="group",        
    height=600
)

fig.show()

la même chose avec :  Films US / Films Français

In [14]:
#US
df_us = df_exploded[df_exploded['ProductionCountries'].str.contains('US', case=False, na=False)]
actor_genre_counts_us = df_us.groupby(['Genres', 'Actors']).size().reset_index(name='Count')
top_actors_per_genre_us = actor_genre_counts_us.sort_values(['Genres', 'Count'], ascending=[True, False])
top_actors_per_genre_us = top_actors_per_genre_us.groupby('Genres').head(5)
print(top_actors_per_genre_us)

         Genres                 Actors  Count
1506     Action           Bruce Willis     46
10156    Action      Samuel L. Jackson     29
10894    Action     Sylvester Stallone     26
5116     Action  Jean-Claude Van Damme     25
8586     Action           Nicolas Cage     25
...         ...                    ...    ...
158295  Western             John Wayne     16
158908  Western               Tim Holt     11
157752  Western         Clint Eastwood      9
158071  Western           Harry Morgan      9
158720  Western         Richard Martin      8

[116 rows x 3 columns]


In [22]:
fig = px.bar(
    data_frame=top_actors_per_genre_us,
    x="Actors",
    y="Count",
    animation_frame= 'Genres',                   
    title="Acteurs les plus présents par genre",
    barmode="group",        
    height=600
)

fig.show()
fig.write_html("../visuals/top_actors_per_genre_us.html")

In [16]:
#FR
df_fr = df_exploded[df_exploded['ProductionCountries'].str.contains('FR', case=False, na=False)]
actor_genre_counts_fr = df_fr.groupby(['Genres', 'Actors']).size().reset_index(name='Count')
top_actors_per_genre_fr = actor_genre_counts_fr.sort_values(['Genres', 'Count'], ascending=[True, False])
top_actors_per_genre_fr = top_actors_per_genre_fr.groupby('Genres').head(5)
print(top_actors_per_genre_fr)

        Genres              Actors  Count
820     Action           Jean Reno      7
843     Action  Jean-Paul Belmondo      7
1624    Action       Sami Bouajila      7
34      Action        Alban Lenoir      6
696     Action    Gérard Depardieu      6
...        ...                 ...    ...
46415  Western          Lex Barker      5
46446  Western        Pierre Brice      4
46457  Western       Rik Battaglia      4
46476  Western        Terence Hill      4
46377  Western        Gustavo Rojo      3

[110 rows x 3 columns]


In [19]:
fig = px.bar(
    data_frame=top_actors_per_genre_fr,
    x="Actors",
    y="Count",
    animation_frame= 'Genres',                   
    title="Acteurs les plus présents par genre",
    barmode="group",        
    height=600,
    range_y=(0,35)
)

fig.show()